# A question of Pham Huu Tiep about a group of symplectic type

* Author: Thomas Breuer
* Version: OSCAR version 1.5
* Last modified: September 21, 2025

Consider the central product E of the cyclic group of order four and the extraspecial group $2^{1+6}_+$, such that $|E| = 2^8$, and let $\Phi$ be one of the two (complex conjugate) faithful irreducible complex representations of degree $8$ of $E$.  The outer automorphism group $A$ of $E$ has the structure $C_2 \times Sp(6, 2)$.

From theory, we know that the representation $\Phi$ extends to a representation of an extension $X = E.B$ of $E$ by the index two subgroup $B$ of $A$.  Moreover, we know that this representation of $X$ can be written over the field $K = Q(\exp(2\pi i/4))$, that is, the normalizer of $\Phi(E)$ in $GL(8, K)$ contains a copy of $X$.

Our task is to show that in this representation, the normalizer in $X$ of a Sylow $7$-subgroup of $X$ contains elements of order $24$ with eigenvalues $\pm z_8$ (each with multiplicity $2$), $\pm z_{24}^{19}$, and $z_{24}^{11}$, where $z_n = \exp(2\pi i/n)$.  We use the computer algebra system [OSCAR](https://oscar.computeralgebra.de).

In [1]:
using Oscar

  ___   ___   ___    _    ____
 / _ \ / __\ / __\  / \  |  _ \  | Combining and extending ANTIC, GAP,
| |_| |\__ \| |__  / ^ \ |  ´ /  | Polymake and Singular
 \___/ \___/ \___//_/ \_\|_|\_\  | Type "?Oscar" for more information
o--------o-----o-----o--------o  | Documentation: https://docs.oscar-system.org
  S Y M B O L I C   T O O L S    | 1.6.0-DEV


We start with the direct product of the cyclic group of order four and $2^{1+6}_{+}$, construct its absolutely irreducible representations, and pick one such representation $\varphi$ of degree $8$ that has kernel of order two. (Note that there are exactly two such representations, which are mutually complex conjugate.) We continue with the matrix group that is the image of $\varphi$, which we identify with $\Phi(E)$.

In [2]:
g1 = cyclic_group(4);
g2 = extraspecial_group(2, 3, :+);
dp = direct_product(g1, g2);
order(dp)

512

In [3]:
irreps = irreducible_modules(CyclotomicField, dp);
dim8 = filter(x -> dim(x) == 8, irreps);
grps = map(m -> matrix_group(map(y -> matrix(y), action(m))), dim8);
filter!(x -> order(x) == 256, grps);
length(grps)

2

In [4]:
E = grps[1];
C = center(E)[1];
trace(cyclic_generator(C))

8*$(4)

By construction, the matrices are monomial and are written over the fourth cyclotomic field. The center of $E$ is cyclic of order four.

In [5]:
F = base_ring(E)

Number field with defining polynomial _$^2 + 1
  over rational field

Next we compute the automorphism group $A$ of $E$, and its derived subgroup $B \cong 2^6.Sp(6, 2) \cong 2^6.O(7,2)$.

In [6]:
aut = automorphism_group(E);
der = derived_subgroup(aut)[1];
describe(der)

"(C2 x C2 x C2 x C2 x C2 x C2) . O(7,2)"

For each generator $\alpha$ of $B$, we want to compute a matrix $M_\alpha$ with the property that $g^\alpha = M_\alpha^{-1} g M_\alpha$ holds for all generators $g$ of $E$.  This yields a linear equation system whose solution determines $M_\alpha$ up to scalar multiples.  The following function determines the solution of the equation system, interprets it as a matrix, and then returns a scalar multiple with determinant one.

In [7]:
function induced_matrix(alpha, E)
  GE = natural_gmodule(E);
  M = GE.M
  GF = gmodule(E, [hom(M, M, matrix(alpha(g))) for g in gens(E)])
  h = Oscar.GModuleFromGap.hom_base(GF, GE)
  @assert length(h) == 1
  d = det(h[1])
  r = root(d, dim(M))
  return h[1] / r
end;

Now we create the group $X$.

In [8]:
autmats = map(alpha -> induced_matrix(alpha, E), gens(der));
GL8F = general_linear_group(8, F);
Xgens = vcat(gens(E), [GL8F(x) for x in autmats]);
X = matrix_group(F, 8, Xgens);
order(X) == 2^8 * order(symplectic_group(6, 2))

true

Finally, we find the requested group element.  The Sylow $7$-normalizer in $X$ has a unique class of subgroups of order $24$, we pick one element and compute its eigenvalues.

In [9]:
syl7 = sylow_subgroup(X, 7)[1];
n = normalizer(X, syl7)[1];
F = quo(n, syl7)[1];
is_cyclic(F)

true

In [10]:
order(F)

24

In [11]:
while true
  global y = rand(n)
  order(y) == 24 && break
end
mat = matrix(y);
mats = [mat^i for i in coprime_residues(24)];
pols = map(characteristic_polynomial, mats);
F24, z24 = cyclotomic_field(24);
i = findfirst( x -> is_zero(x(z24^11)), pols );
pol24 = change_base_ring(F24, pols[i]);
factor(pol24)

(x - z_24^7) * (x + z_24^3)^2 * (x + z_24^7 - z_24^3) * (x + z_24^7) * (x - z_24^3)^2 * (x - z_24^7 + z_24^3)

The eigenvalues are $\pm z_8 = \pm z_{24}^3$ (each with multiplicity $2$), $\pm z_{24}^7 = \pm z_{24}^{19}$, and $\pm (z_{24}^7 - z_{24}^3) = \pm z_8 z_3 = \pm z_{24}^{11}$, as requested.